In [1]:
# Imports

# Data import and manipulation
import pandas as pd
# Math
import numpy as np
# Let's go ahead and seed the notebook, for reproducibility
np.random.seed(113)

# Visualizing
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as offline

# Interactive widget
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
df = pd.read_csv("data/TX_Buyers_Shipment_Location_Data.csv",
                 usecols=["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
                          "BUYER_NAME", "BUYER_LAT", "BUYER_LONG",
                          "TRANSACTION_DATE", "DOSAGE_UNIT"])
df.head()

FileNotFoundError: [Errno 2] File b'data/TX_Buyers_Shipment_Location_Data.csv' does not exist: b'data/TX_Buyers_Shipment_Location_Data.csv'

In [3]:
df["TRANSACTION_DATE"] = pd.to_datetime(df["TRANSACTION_DATE"],
                                          format='%Y-%m-%d')

In [4]:
df["TRANSACTION_YEAR"] = df["TRANSACTION_DATE"].dt.year

In [5]:
df.drop(columns="TRANSACTION_DATE", inplace=True)

In [6]:
data_agg = df.groupby(["REPORTER_NAME", "REPORTER_LAT", "REPORTER_LONG",
                       "BUYER_NAME", "BUYER_LAT", "BUYER_LONG",
                       "TRANSACTION_YEAR"]).agg({"DOSAGE_UNIT": "sum"}).reset_index()

In [7]:
data_agg.head()

,REPORTER_NAME,REPORTER_LAT,REPORTER_LONG,BUYER_NAME,BUYER_LAT,BUYER_LONG,TRANSACTION_YEAR,DOSAGE_UNIT
0,"A F HAUSER, INC",41.5,-87.0,"ADAMS, NATHALIA C (DVM)",33.23,-97.11,2006,5000.0
1,"A F HAUSER, INC",41.5,-87.0,"ALLRED, JOHN A",29.70,-98.10,2006,1900.0
2,"A F HAUSER, INC",41.5,-87.0,"BOBECHKO, WALTER P MD",32.90,-96.79,2006,1500.0
3,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2009,700.0
4,"A F HAUSER, INC",41.5,-87.0,"BRANTLEY, GARY C DVM",32.97,-96.74,2010,200.0


In [8]:
data_agg.shape

(85658, 8)

In [9]:
data_agg["DOSAGE_UNIT"].describe()

count    8.565800e+04
mean     6.341626e+04
std      1.285731e+05
min      1.000000e+00
25%      1.400000e+03
50%      1.046000e+04
75%      7.013000e+04
max      3.830800e+06
Name: DOSAGE_UNIT, dtype: float64

In [24]:
data_agg['TEXT'] = data_agg['BUYER_NAME'] + '<br>Total Pills ' + (data_agg['DOSAGE_UNIT']).astype(str)

In [26]:
# data_agg.to_csv("data/TX_County_Buyer_Data_Aggregated.csv", index=False)

In [10]:
years = list(range(2006,2013))
years

[2006, 2007, 2008, 2009, 2010, 2011, 2012]

In [11]:
# Using the ipywidgets
@interact
# Defining our visualization function
# Here, the init values in the function define the dropdown menus
def track_shipments(year=years):

    # Creating a dataframe based on the year chosen
    df = data_agg.loc[data_agg["TRANSACTION_YEAR"] == year]

    fig = go.Figure()

    fig.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=df['BUYER_LONG'],
        lat=df['BUYER_LAT'],
        text=df['BUYER_NAME'],
        hoverinfo='text',
        mode='markers',
        marker=dict(
            size=df["DOSAGE_UNIT"] / 100000,
            color=df["DOSAGE_UNIT"],
            colorscale="Viridis",
            line=dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        )))

    fig.update_layout(
        title_text="Received Opioid Shipments in Texas, " + str(year),
        showlegend=False,
        geo=go.layout.Geo(
            scope='usa',
            projection=go.layout.geo.Projection(
                type='albers usa',
                scale=2.3
            ),
            center={'lat': 31.9686, 'lon': -99.9018},
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',

        )
    )

    fig.show()
    
#     offline.plot(fig, filename='opioids_received_TX'+'.html') 

interactive(children=(Dropdown(description='year', options=(2006, 2007, 2008, 2009, 2010, 2011, 2012), value=2…

In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output 

In [13]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash("Opioid_Data_Visualizing_Buyer_Locations",
                external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=data_agg["TRANSACTION_YEAR"].min(),
        max=data_agg["TRANSACTION_YEAR"].max(),
        value=data_agg["TRANSACTION_YEAR"].min(),
        marks={str(year): str(year)
               for year in data_agg["TRANSACTION_YEAR"].unique()},
        step=None
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    [Input('year-slider', 'value')])
def update_figure(selected_year):
    filtered_df = data_agg.loc[data_agg["TRANSACTION_YEAR"] == selected_year]
    traces = go.Scattergeo(
        locationmode='USA-states',
        lon=df['BUYER_LONG'],
        lat=df['BUYER_LAT'],
        text=df['BUYER_NAME'],
        hoverinfo='text',
        mode='markers',
        marker=dict(
            size=df["DOSAGE_UNIT"] / 100000,
            color=df["DOSAGE_UNIT"],
            colorscale="Viridis",
            line=dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            title_text="Received Opioid Shipments in Texas, " + str(year),
            showlegend=False,
            geo=go.layout.Geo(
                scope='usa',
                projection=go.layout.geo.Projection(
                    type='albers usa',
                    scale=2.3
                ),
                center={'lat': 31.9686, 'lon': -99.9018},
                showland=True,
                landcolor='rgb(243, 243, 243)',
                countrycolor='rgb(204, 204, 204)',
            )
        )
    }

In [18]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash("Opioid_Data_Visualizing_Buyer_Locations",
                external_stylesheets=external_stylesheets)

datamap = go.Data([go.Scattergeo(
    lat = ['38.91427', '38.91538', '38.91458',
        '38.92239', '38.93222', '38.90842',
        '38.91931', '38.93260', '38.91368',
        '38.88516', '38.921894', '38.93206',
        '38.91275'
    ],
    lon = ['-77.02827', '-77.02013', '-77.03155',
        '-77.04227', '-77.02854', '-77.02419',
        '-77.02518', '-77.03304', '-77.04509',
        '-76.99656', '-77.042438', '-77.02821',
        '-77.01239'
    ],
    mode = 'markers+lines',
    line = dict(width = 2),
    marker = go.Marker(
        size = 11
      )
    )])

layoutmap = go.Layout(
    height = 700,
    autosize = True,
    showlegend = False,
    hovermode = 'closest',
    geo = dict(
        projection = dict(type = "equirectangular"),
        ),
    mapbox = dict(
        bearing = 1,
        center = dict(
            lat = 38.92,
            lon = -77.07
            ),
        pitch = 0,
        zoom = 5,
    ),
)

fig = dict( data=datamap, layout=layoutmap )  


app.layout = html.Div([dcc.Graph(id='graph', figure=fig)])